In [1]:
import cv2
import numpy as np
 

In [2]:
largeur_min_contour = 40  
longeur_min_contour = 40  
offset = 10  
ligne_hauteur = 550  
matches = []
vehicles = 0
#les conditin poses

In [3]:
def get_centrolid(x, y, w, h):
   x1 = int(w / 2)
   #on divise la largeur sur 2
   y1 = int(h / 2)
   #on divise la hauteur sur 2
   cx = x + x1
   cy = y + y1
   return cx, cy
#La fonction calcule le point qui se trouve au centre du rectangle

In [8]:
cap = cv2.VideoCapture('Video.mp4')
#ici on lit la video

In [9]:
cap.set(3, 1920)
cap.set(4, 1080)
#definir la resolution de la video
 
if cap.isOpened():
   #Cette condition verifie si la capture vidéo est ouverte avec succes
   ret, frame1 = cap.read()
   #Si la capture video est ouverte cette ligne lit la première image de la vidéo et la stocke dans la variable frame1.
   #ret c'est un booelan qui indiquant si la lecture de l'image s'est bien passée
else:
   ret = False
ret, frame1 = cap.read()
ret, frame2 = cap.read()
# on lits deux images supplementaires de la vidéo et on les stocks dans les variables frame1 et frame2 meme si la captre video n'est pas ouverte

In [10]:
while ret:
   d = cv2.absdiff(frame1, frame2)
   #cette fonction calcule la difference entre les 2 frame en valeur absolue
   #cette fonction donne des pixels eleve indiquent des changements significatifs, tandis que des valeurs faibles  indiquent peu ou pas de changement.
   
   gris = cv2.cvtColor(d, cv2.COLOR_BGR2GRAY)
   #convertit l'image d en niveaux de gris 
    
   blur = cv2.GaussianBlur(gris, (5, 5), 0)
   #appliquer le filtre gaussian pour elimine le bruit et  il peut rendre les contours plus  facile a detecter il amelior la visibiliter des contours
   #un noyau de dimension (5, 5) est utilisé
   
   ret, th = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
   #la fonction effectue une binarisation de l'image floutee tel que les valeur moin de 20 soit 0 et 255 est la valeur max des pixels
   
   dilated = cv2.dilate(th, np.ones((3, 3)))
   # utilisation de la fonction est pour appliquer une opération de dilatation à l'image binaire pour connecter les régions blanches disjointes dans une image binaire
   #améliorer la continuite des regions et simplifier le processus de détection des contours
   
   kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2))
   #creer element structurant qui sera utilise dans des opérations de traitement d'image
   
   fermiture = cv2.morphologyEx(dilated, cv2.MORPH_CLOSE, kernel)
   #applique une operation morphologique de fermeture à l'image dilatee La fermeture est une combinaison d'une dilatation suivie d'une érosion
   #La fermeture est utilisée pour fermer les petits trous à l'intérieur des objets blancs et pour connecter des objets blancs distincts
   
   contours, h = cv2.findContours(
       fermiture, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
   #trouver les contours dans l'image binaire résultante après l'opération morphologique de fermeture 
   
   for(i, c) in enumerate(contours):
    #parcourir les contours detecter par l'image
       (x, y, w, h) = cv2.boundingRect(c)
       #cette fonctin renvoie les coordonne du rectangle qui englobe le contour 
       contour_valid = (w >= largeur_min_contour) and (
           h >= longeur_min_contour)
       #on virifie la validite des contours par rapport aux condition qu'on a pose au debut
    
       if not contour_valid:
           continue
       cv2.rectangle(frame1, (x-10, y-10), (x+w+10, y+h+10), (0, 0, 255), 2)
       #dessiner les rectangle des contour valides on utilison les coordone trouver 
 
       cv2.line(frame1, (0, ligne_hauteur), (1200, ligne_hauteur), (0, 255, 0), 2)
       #pour dessiner la ligne
       
       centrolid = get_centrolid(x, y, w, h)
       #calcule du centre qui englobe le rectangle
       matches.append(centrolid)
       #ajouter les valeurs a la liste matches
       
       cv2.circle(frame1, centrolid, 5, (0, 255, 0), -1)
       #dessiner un cercle autour du centre
       cx, cy = get_centrolid(x, y, w, h)
       for (x, y) in matches:
           if y < (ligne_hauteur+offset) and y > (ligne_hauteur-offset):
               vehicles = vehicles +1
               matches.remove((x, y))
        #verifier si le centrolid a traverser la ligne et calculer le nombre de veicule pui supprime le centroide
   
   
   cv2.putText(frame1, "Nombre total detecter : " + str(vehicles), (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1,
               (0, 170, 0), 2)
   #ecrirce le text sur la video
 
 
 
   cv2.imshow("Voiture Detection", frame1)
   if cv2.waitKey(1) == 27:
       break
   frame1 = frame2
   ret, frame2 = cap.read()
   #voir la video a appuyer sur echap pour sortir de la video
cv2.destroyAllWindows()
cap.release()

### On voit maintenant l'impact des  differents fonctions  

In [11]:
cv2.imshow("utiliser la fonction absdiff entre 2 frame", frame1)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [12]:
cv2.imshow("utiliser la fonction absdiff entre 2 frame", d)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [13]:
cv2.imshow("rendre l'image noir est blanc", gris)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [14]:
cv2.imshow("guassian filter", blur)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [15]:
cv2.imshow("binarisation", th)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [16]:
cv2.imshow("dilatation", dilated)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [17]:
cv2.imshow("combler les petit troux dans la ragion blanche", fermiture)
cv2.waitKey(0)
cv2.destroyAllWindows()